# Accelerating End-to-End Data Science Workflows # 

## 03 - การจัดการหน่วยความจำ (Memory Management) ##

**สารบัญ**

โน้ตบุ๊กนี้จะสำรวจพลวัตระหว่างข้อมูลและหน่วยความจำ โน้ตบุ๊กนี้ครอบคลุมส่วนต่าง ๆ ดังนี้:

1.  [การจัดการหน่วยความจำ (Memory Management)](#Memory-Management)
    * [การใช้หน่วยความจำ (Memory Usage)](#Memory-Usage)
2.  [ชนิดข้อมูล (Data Types)](#Data-Types)
    * [แปลงชนิดข้อมูล (Convert Data Types)](#Convert-Data-Types)
    * [แบบฝึกหัดที่ 1 - แก้ไข `dtypes`](#Exercise-#1---Modify-dtypes)
    * [ข้อมูลประเภท Categorical](#Categorical)
3.  [การโหลดข้อมูลอย่างมีประสิทธิภาพ (Efficient Data Loading)](#Efficient-Data-Loading)


## การจัดการหน่วยความจำ (Memory Management) ##

ในระหว่างกระบวนการได้มาซึ่งข้อมูล ข้อมูลจะถูกถ่ายโอนไปยังหน่วยความจำเพื่อให้โปรเซสเซอร์ดำเนินการ การจัดการหน่วยความจำมีความสำคัญอย่างยิ่งต่อ cuDF และการทำงานของ GPU ด้วยเหตุผลสำคัญหลายประการ:

* **หน่วยความจำ GPU ที่จำกัด**: โดยทั่วไป GPU มีหน่วยความจำน้อยกว่า CPU ดังนั้นการจัดการหน่วยความจำที่มีประสิทธิภาพจึงเป็นสิ่งจำเป็นเพื่อเพิ่มการใช้หน่วยความจำ GPU ที่มีอยู่ให้สูงสุด โดยเฉพาะสำหรับชุดข้อมูลขนาดใหญ่
* **ค่าใช้จ่ายในการถ่ายโอนข้อมูล**: การถ่ายโอนข้อมูลระหว่างหน่วยความจำ CPU และ GPU ค่อนข้างช้าเมื่อเทียบกับความเร็วในการคำนวณของ GPU การลดการถ่ายโอนเหล่านี้ผ่านการจัดการหน่วยความจำที่ชาญฉลาดเป็นสิ่งสำคัญต่อประสิทธิภาพ
* **การปรับแต่งประสิทธิภาพ**: การทำความเข้าใจและเพิ่มประสิทธิภาพการใช้หน่วยความจำเป็นกุญแจสำคัญในการบรรลุประสิทธิภาพสูงสุดในงานประมวลผลข้อมูลที่เร่งความเร็วด้วย GPU

เมื่อทำได้อย่างถูกต้อง การเก็บข้อมูลไว้บน GPU สามารถช่วยให้ cuDF และระบบนิเวศ RAPIDS บรรลุประสิทธิภาพที่ดีขึ้นอย่างมาก จัดการชุดข้อมูลขนาดใหญ่ขึ้น และมอบความสามารถในการประมวลผลข้อมูลที่มีประสิทธิภาพมากขึ้น

ด้านล่างนี้เราจะนำเข้าข้อมูลจากไฟล์ csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# DO NOT CHANGE THIS CELL
import pandas as pd
import random
import time

In [ ]:
# CHANGE THIS CELL
df=pd.read_csv('./data/uk_pop.csv')

# preview
df.head()

### การใช้หน่วยความจำ (Memory Usage) ###

การใช้หน่วยความจำของ DataFrame ขึ้นอยู่กับชนิดข้อมูลของแต่ละคอลัมน์

<p><img src='./images/dtypes.png' width=720></p>

เราสามารถใช้ `DataFrame.memory_usage()` เพื่อดูการใช้หน่วยความจำสำหรับแต่ละคอลัมน์ (เป็นไบต์) ชนิดข้อมูลทั่วไปส่วนใหญ่มีขนาดคงที่ในหน่วยความจำ เช่น `int`, `float`, `datetime` และ `bool` การใช้หน่วยความจำสำหรับชนิดข้อมูลเหล่านี้คือความต้องการหน่วยความจำที่เกี่ยวข้องคูณด้วยจำนวนจุดข้อมูล สำหรับชนิดข้อมูล `string` การใช้หน่วยความจำที่รายงาน _สำหรับ pandas_ คือจำนวนองค์ประกอบคูณด้วย 8 ไบต์ สิ่งนี้คิดเป็น 64 บิตที่จำเป็นสำหรับพอยน์เตอร์ที่ชี้ไปยังที่อยู่หน่วยความจำ แต่ไม่ใช่หน่วยความจำที่ใช้สำหรับค่าสตริงจริง หน่วยความจำจริงที่จำเป็นสำหรับค่าสตริงคือ 49 ไบต์บวกกับอีกหนึ่งไบต์สำหรับแต่ละตัวอักษร พารามิเตอร์ `deep` ให้รายงานการใช้หน่วยความจำที่แม่นยำยิ่งขึ้นซึ่งคิดถึงการใช้หน่วยความจำระดับระบบของชนิดข้อมูล `string` ที่บรรจุอยู่

ด้านล่างนี้ เราจะดูการใช้หน่วยความจำ

In [ ]:
# DO NOT CHANGE THIS CELL
# pandas memory utilization
mem_usage_df=df.memory_usage(deep=True)
mem_usage_df

ด้านล่างนี้ เรากำหนดฟังก์ชัน `make_decimal()` เพื่อแปลงขนาดหน่วยความจำให้เป็นหน่วยตามกำลังของ 2 ซึ่งแตกต่างจากหน่วยตามกำลังของ 10 แต่เป็นธรรมเนียมปฏิบัติที่นิยมใช้ในการรายงานความจุหน่วยความจำ สามารถดูข้อมูลเพิ่มเติมเกี่ยวกับสองนิยามได้ [ที่นี่](https://en.wikipedia.org/wiki/Byte#Multiple-byte_units)

In [ ]:
# DO NOT CHANGE THIS CELL
suffixes = ['B', 'kB', 'MB', 'GB', 'TB', 'PB']
def make_decimal(nbytes):
    i=0
    while nbytes >= 1024 and i < len(suffixes)-1:
        nbytes/=1024.
        i+=1
    f=('%.2f' % nbytes).rstrip('0').rstrip('.')
    return '%s %s' % (f, suffixes[i])

In [ ]:
make_decimal(mem_usage_df.sum())

ด้านล่างนี้ เราจะคำนวณการใช้หน่วยความจำด้วยตนเองโดยอิงตามชนิดข้อมูล

In [ ]:
# DO NOT CHANGE THIS CELL
# get number of rows
num_rows=len(df)

# 64-bit numbers uses 8 bytes of memory
print(f'Numerical columns use {num_rows*8} bytes of memory')

In [ ]:
# DO NOT CHANGE THIS CELL
# check random string-typed column
string_cols=[col for col in df.columns if df[col].dtype=='object' ]
column_to_check=random.choice(string_cols)

overhead=49
pointer_size=8

# nan==nan when value is not a number
# nan uses 32 bytes of memory
string_col_mem_usage_df=df[column_to_check].map(lambda x: len(x)+overhead+pointer_size if x else 32)
string_col_mem_usage=string_col_mem_usage_df.sum()
print(f'{column_to_check} column uses {string_col_mem_usage} bytes of memory.')

**หมายเหตุ**: ชนิดข้อมูล `string` ถูกจัดเก็บต่างกันใน cuDF และ pandas สามารถดูข้อมูลเพิ่มเติมเกี่ยวกับวิธีการที่ `libcudf` จัดเก็บข้อมูลสตริงโดยใช้ [Arrow format](https://arrow.apache.org/docs/format/Columnar.html#variable-size-binary-layout) ได้ [ที่นี่](https://developer.nvidia.com/blog/mastering-string-transformations-in-rapids-libcudf/)

## ชนิดข้อมูล (Data Types) ##

โดยค่าเริ่มต้น pandas (และ cuDF) ใช้ตัวเลขขนาด 64 บิตสำหรับการจัดเก็บค่าตัวเลข การใช้ตัวเลข 64 บิตให้ความแม่นยำสูงสุด แต่แอปพลิเคชันจำนวนมากไม่ต้องการความแม่นยำ 64 บิตเมื่อทำการรวมข้อมูลจำนวนมาก เมื่อเป็นไปได้ การใช้ตัวเลข 32 บิตจะช่วยลดความต้องการพื้นที่จัดเก็บและหน่วยความจำลงครึ่งหนึ่ง และโดยทั่วไปจะช่วยเร่งการคำนวณอย่างมากเนื่องจากข้อมูลที่ต้องเข้าถึงในหน่วยความจำลดลงครึ่งหนึ่ง

### การแปลงชนิดข้อมูล (Convert Data Types) ###

เมธอด `.astype()` สามารถใช้เพื่อแปลงชนิดข้อมูลตัวเลขให้ใช้คอนเทนเนอร์ขนาดบิตที่แตกต่างกัน ที่นี่เราจะแปลงคอลัมน์ `age` จาก `int64` เป็น `int8`

In [ ]:
# DO NOT CHANGE THIS CELL
df['age']=df['age'].astype('int8')

df.dtypes

### แบบฝึกหัดที่ 1 - แก้ไข `dtypes` ###

**คำแนะนำ**:
* แก้ไขเฉพาะส่วน `<FIXME>` เท่านั้น และรันเซลล์ด้านล่างเพื่อแปลงชนิดข้อมูล 64 บิตใดๆ ให้เป็นชนิดข้อมูล 32 บิตที่เทียบเท่า

In [ ]:
df[<<<<FIXME>>>>]=df[<<<<FIXME>>>>].astype('float32')
df[<<<<FIXME>>>>]=df[<<<<FIXME>>>>].astype('float32')

Click ... for solution. 

### Categorical ###

ข้อมูลประเภท Categorical คือชนิดข้อมูลที่แสดงถึงหมวดหมู่หรือกลุ่มที่แยกจากกันอย่างชัดเจน สามารถมีลำดับหรืออันดับที่มีความหมายได้ แต่โดยทั่วไปไม่สามารถใช้สำหรับการดำเนินการทางตัวเลขได้ เมื่อเหมาะสม การใช้ชนิดข้อมูล `categorical` สามารถลดการใช้หน่วยความจำและนำไปสู่การดำเนินการที่เร็วขึ้นได้ นอกจากนี้ยังสามารถใช้เพื่อกำหนดและรักษาระเบียบของหมวดหมู่ที่กำหนดเองได้

ด้านล่างนี้ เราจะหาจำนวนค่าที่ไม่ซ้ำกันในคอลัมน์สตริง

In [ ]:
# DO NOT CHANGE THIS CELL
df.select_dtypes(include='object').nunique()

ด้านล่างนี้ เราจะแปลงคอลัมน์ที่มีค่าที่ไม่ต่อเนื่องจำนวนน้อยให้เป็นชนิด `category` ชนิดข้อมูล `category` มีคุณสมบัติ `.categories` และ `.codes` ซึ่งสามารถเข้าถึงได้ผ่านทาง `.cat`

In [ ]:
# DO NOT CHANGE THIS CELL
df['sex']=df['sex'].astype('category')
df['county']=df['county'].astype('category')

In [ ]:
# DO NOT CHANGE THIS CELL
display(df['county'].cat.categories)
print('-'*40)
display(df['county'].cat.codes)

**หมายเหตุ**: `.astype()` ยังสามารถใช้เพื่อแปลงข้อมูลเป็น `datetime` หรือ `object` เพื่อเปิดใช้งานเมธอดสำหรับ datetime และสตริง


## การโหลดข้อมูลอย่างมีประสิทธิภาพ (Efficient Data Loading) ##

บ่อยครั้งที่การระบุชนิดข้อมูลที่เหมาะสมที่สุดสำหรับแต่ละคอลัมน์นั้นมีประโยชน์ โดยพิจารณาจากช่วงของข้อมูล ความแม่นยำที่ต้องการ และวิธีการนำข้อมูลไปใช้

In [ ]:
# DO NOT CHANGE THIS CELL
start=time.time()
df=pd.read_csv('./data/uk_pop.csv')
duration=time.time()-start

mem_usage_df=df.memory_usage(deep=True)
display(mem_usage_df)

print(f'Loading {make_decimal(mem_usage_df.sum())} took {round(duration, 2)} seconds.')

ด้านล่างนี้ เราจะเปิดใช้งาน `cuda.pandas` เพื่อดูความแตกต่าง


In [ ]:
# DO NOT CHANGE THIS CELL
%load_ext cudf.pandas

import pandas as pd
import time

In [ ]:
# DO NOT CHANGE THIS CELL
suffixes = ['B', 'kB', 'MB', 'GB', 'TB', 'PB']
def make_decimal(nbytes):
    i=0
    while nbytes >= 1024 and i < len(suffixes)-1:
        nbytes/=1024.
        i+=1
    f=('%.2f' % nbytes).rstrip('0').rstrip('.')
    return '%s %s' % (f, suffixes[i])

In [ ]:
%%cudf.pandas.line_profile
# DO NOT CHANGE THIS CELL
start=time.time()

# define data types for each column
dtype_dict={
    'age': 'int8', 
    'sex': 'category', 
    'county': 'category', 
    'lat': 'float64', 
    'long': 'float64', 
    'name': 'category'
}
        
efficient_df=pd.read_csv('./data/uk_pop.csv', dtype=dtype_dict)
duration=time.time()-start

mem_usage_df=efficient_df.memory_usage('deep')
display(mem_usage_df)

print(f'Loading {make_decimal(mem_usage_df.sum())} took {round(duration, 2)} seconds.')


เราสามารถโหลดข้อมูลได้เร็วขึ้นและมีประสิทธิภาพมากขึ้น

**หมายเหตุ**: สังเกตว่าหน่วยความจำที่ใช้บน GPU มีขนาดใหญ่กว่าหน่วยความจำที่ DataFrame ใช้ นี่เป็นสิ่งที่คาดไว้ เนื่องจากการโหลดข้อมูลมีกระบวนการตัวกลางที่ใช้หน่วยความจำบางส่วนในระหว่างกระบวนการโหลดข้อมูล โดยเฉพาะอย่างยิ่งที่เกี่ยวข้องกับการแยกวิเคราะห์ไฟล์ csv ในกรณีนี้
```
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1B.0 Off |                    0 |
| N/A   32C    P0    26W /  70W |   1378MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:1C.0 Off |                    0 |
| N/A   31C    P0    26W /  70W |    168MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   2  Tesla T4            Off  | 00000000:00:1D.0 Off |                    0 |
| N/A   30C    P0    26W /  70W |    168MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   3  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   30C    P0    26W /  70W |    168MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU   GI   CI        PID   Type   Process name                  GPU Memory |
|        ID   ID                                                   Usage      |
|=============================================================================|
+-----------------------------------------------------------------------------+
```

In [ ]:
# DO NOT CHANGE THIS CELL
!nvidia-smi

เมื่อโหลดข้อมูลด้วยวิธีนี้ เราอาจสามารถใส่ข้อมูลได้มากขึ้น ขนาดชุดข้อมูลที่เหมาะสมที่สุดขึ้นอยู่กับปัจจัยหลายประการ รวมถึงการดำเนินการเฉพาะที่กำลังทำอยู่ ความซับซ้อนของภาระงาน และหน่วยความจำ GPU ที่มีอยู่ เพื่อเพิ่มการเร่งความเร็ว ชุดข้อมูลควรพอดีกับหน่วยความจำ GPU โดยมีพื้นที่เพียงพอสำหรับสำหรับการดำเนินการที่อาจเพิ่มความต้องการหน่วยความจำ โดยทั่วไปแล้ว cuDF แนะนำชุดข้อมูลที่มีขนาดน้อยกว่า 50% ของความจุหน่วยความจำ GPU

In [ ]:
# DO NOT CHANGE THIS CELL
# 1 gigabytes = 1073741824 bytes
mem_capacity=16*1073741824

mem_per_record=mem_usage_df.sum()/len(efficient_df)

print(f'We can load {int(mem_capacity/2/mem_per_record)} number of rows.')

In [ ]:
# DO NOT CHANGE THIS CELL
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

**เยี่ยมมาก!** ไปยัง [โน้ตบุ๊กถัดไป](1-04_interoperability.ipynb) กันเลย